In [6]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import nltk
from nltk.tokenize import RegexpTokenizer

In [7]:
df_all = pd.read_excel(r'D:\TEST Codingan\Sistem-Temu-Kembali-Informasi\Riset Jupiter Notebook\Data\df_final.xlsx')

# EDA

In [8]:
df_all = df_all.drop(columns = ['Unnamed: 0'])

In [4]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   divisions          917 non-null    object
 1   title              1000 non-null   object
 2   abstract           758 non-null    object
 3   subjects           1000 non-null   object
 4   publication        498 non-null    object
 5   publisher          863 non-null    object
 6   creators           999 non-null    object
 7   date               1000 non-null   object
 8   keywords           181 non-null    object
 9   uri                1000 non-null   object
 10  new_abstract       758 non-null    object
 11  text_raw           1000 non-null   object
 12  language           1000 non-null   object
 13  language_accuracy  1000 non-null   object
 14  fulltext           1000 non-null   object
 15  fulltext_stopword  1000 non-null   object
 16  text_stemmed       1000 non-null   object
d

In [5]:
pd.DataFrame(df_all[['title', 'abstract']])

,title,abstract
0,TOKSISITAS METIL METSULFURON HUBUNGANNYA DENGA...,Metil metsulfuron merupakan senyawa aktif yang...
1,Media Pembelajaran Digital sebagai Sumber Bela...,Kegiatan pembelajaran siswa SMA dan SMK mutlak...
2,Strategi Keberlanjutan Jasa Air Bersih Berbasi...,Persoalan penyediaan jasa air bersih merupakan...
3,Metode Sexing Spermatozoa XY dan Aplikasi Inse...,NaN
4,PENGARUH BAHAN PELLETING TERHADAP PERKECAMBAHA...,Bawang merah merupakan komoditas pertanian pen...
...,...,...
995,Analisis Kelayakan Finansial Usaha Burung Puyu...,Unggas penghasil telur terbesar kedua setelah ...
996,Buku Teknologi Pengelolaan Agen Biologis Hara,NaN
997,Teknologi Pengelolaan Agen Biologis Hara (TPABH),NaN
998,PENGARUH KECERDASAN EMOSIONAL DAN KEMAMPUAN AW...,Pembelajaran online menyisakan banyak permasal...


In [ ]:
topik_riset = {'phew'}

for i in range(len(df_all['fokus_riset'])):
  topik_riset.add(df_all['fokus_riset'].iloc[i])

In [ ]:
topik_riset

{'Integrasi Fokus Riset Energi-Energi Baru dan Terbarukan',
 'Kebencanaan',
 'Kemaritiman',
 'Kesehatan-Obat',
 'Material Maju',
 'Pangan-Pertanian',
 'Pertahanan dan Keamanan',
 'Sosial Humaniora, Seni Budaya, Pendidikan',
 'Teknologi Informasi dan Komunikasi',
 'Transportasi',
 nan,
 'phew'}

In [ ]:
temporal = []

for i in range(len(df_all['fokus_riset'])):
  if df_all['fokus_riset'].iloc[i] == 'Kesehatan-Obat':
    temporal.append(df_all.iloc[i])

In [ ]:
temporal = []

for i in range(len(df_all['fokus_riset'])):
  if pd.isnull(df_all['fokus_riset'].iloc[i]):
    temporal.append(df_all.iloc[i])

In [ ]:
len(temporal)

1240

In [ ]:
df_temp = pd.DataFrame(temporal)
df_temp[['title', 'fokus_riset']]

In [ ]:
df_all['fokus_riset'].iloc[3921] = 'Kesehatan-Obat'
df_all['fokus_riset'].iloc[3926] = 'Sosial Humaniora, Seni Budaya, Pendidikan'

# Mesin Pencari

In [9]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15867 entries, 0 to 15866
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   divisions          14133 non-null  object
 1   title              15867 non-null  object
 2   abstract           12295 non-null  object
 3   subjects           15864 non-null  object
 4   publication        6874 non-null   object
 5   publisher          12213 non-null  object
 6   creators           15851 non-null  object
 7   date               12316 non-null  object
 8   keywords           3418 non-null   object
 9   uri                15867 non-null  object
 10  new_abstract       12295 non-null  object
 11  text_raw           15867 non-null  object
 12  language           15867 non-null  object
 13  language_accuracy  15867 non-null  object
 14  fulltext           15867 non-null  object
 15  fulltext_stopword  15867 non-null  object
 16  text_stemmed       15867 non-null  objec

In [9]:
df_all['title'].iloc[5]

'PENGENALAN BUNGA KRISAN POTONG DI DESA SUNGAI \nLANGKA KAB. PESAWARAN SEBAGAI KOMODITAS BARU DI \nDESA AGROWISATA'

In [15]:
# Inisialisasi TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

In [11]:
# Contoh teks
text1 = 'Tingkat METIL METSULFURON dalam COpedoda dhipa sp hahahaha'
text2 = 'Tingkat METIL METSULFURON dalam TOKSISITAS MASKULINITAS COPEPODA Daphnia sp.'

# Inisialisasi TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Transformasi teks menjadi vektor TF-IDF
tfidf_matrix = tfidf_vectorizer.fit_transform([text1, text2])

# Hitung similaritas kosinus antara kedua teks
cosine_similarities = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])

# Tampilkan hasil similaritas
print("Similaritas antara teks 1 dan teks 2:", cosine_similarities[0][0])

Similaritas antara teks 1 dan teks 2: 0.42113589132819584


In [10]:
dicari = 'hutan kawasan lindung'
dicari = dicari.lower()

In [11]:
def remove_stopwords(text):
    factory = StopWordRemoverFactory()
    stopwords = factory.get_stop_words()

    result = []
    for x in  text.split():
        if x not in stopwords:
            result.append(x)

    hasil_stopword = " ".join(result)

    return hasil_stopword

In [12]:
def stem(text):
    factory2 = StemmerFactory()
    stemmer = factory2.create_stemmer()
    return stemmer.stem(text)

In [13]:
dicari = stem(dicari)
dicari

'hutan kawasan lindung'

In [16]:
from tqdm import tqdm

df_temu = []
similaritas = []

for i in tqdm(range(len(df_all['text_stemmed']))):
  temp = df_all['text_stemmed'].iloc[i]

  # Transformasi teks menjadi vektor TF-IDF
  tfidf_matrix = tfidf_vectorizer.fit_transform([dicari, temp])

  # Hitung similaritas kosinus antara kedua teks
  cosine_similarities = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])

  if cosine_similarities[0][0] > 0.1:
    df_temu.append(df_all.iloc[i])
    similaritas.append(cosine_similarities)

df_temu = pd.DataFrame(df_temu)
df_temu['similaritas'] = similaritas

  0%|          | 0/15867 [00:00<?, ?it/s]

100%|██████████| 15867/15867 [01:04<00:00, 245.28it/s]


In [17]:
len(df_temu)

480

In [18]:
df_temu[['title', 'similaritas','abstract']].sort_values(by=['similaritas'], ascending=False).head(10)

,title,similaritas,abstract
2704,KEANEKARAGAMAN JENIS POHON SEBAGAI SALAH SATU ...,[[0.48976702303714875]],Hutan lindung menjadi suatu kawasan hutan deng...
15796,Analisis Pengelolaan Hutan Kemasyarakatan di S...,[[0.4550747482482657]],Penilaian Rekan Sejawat
1984,ANALISIS PERBANDINGAN PELAKSANAAN HUTAN KEMASY...,[[0.42982552079582437]],Perhutanan Sosial merupakan pengelolaan hutan ...
15386,Nilai Manfaat Hutan Lindung Register 21 Perint...,[[0.4237327171150527]],Intisari — Hutan lindung dapat memberikan manf...
6208,"PEER REVIEW RESULT OF PAPER ""Penilaian Kesehat...",[[0.4006549252121726]],"Di Provinsi Lampung, kesadaran tentang penting..."
3677,Keragaman Spesies Pohon sebagai Karakteristik ...,[[0.39994801257800494]],Lanskap dikenal dengan karakteristik bentang a...
14743,Penilaian Kesehatan Hutan pada Berbagai Tipe H...,[[0.39743254583726817]],"Di Provinsi Lampung, kesadaran tentang penting..."
6211,Penilaian Kesehatan Hutan pada Berbagai Tipe H...,[[0.39743254583726817]],"Di Provinsi Lampung, kesadaran tentang penting..."
6210,Penilaian Kesehatan Hutan pada Berbagai Tipe H...,[[0.3971589668544352]],"ABSTRAK\nDi Provinsi Lampung, kesadaran tentan..."
10191,ANALISIS PENGELOLAAN HUTAN KEMASYARAKATAN DI S...,[[0.3868936119884499]],Hasil Penilaian Teman Sejawat


In [20]:
tokenizer = RegexpTokenizer(r'\w+')

topik = []

for index in range(len(df_temu)):
  teks = df_temu['fulltext_stopword'].iloc[index]
  data = tokenizer.tokenize(teks)

  # Proses vektorisasi teks dengan TF-IDF
  tfidf_vectorizer = TfidfVectorizer()
  tfidf_matrix = tfidf_vectorizer.fit_transform(data)

  # Terapkan LSA dengan TruncatedSVD
  if len(data) <= 5:
     num_topics = len(data) # Jumlah topik yang ingin diidentifikasi
  else:
    num_topics = 5  # Jumlah topik yang ingin diidentifikasi
  lsa_model = TruncatedSVD(n_components=num_topics, random_state=42)
  lsa_model.fit_transform(tfidf_matrix)

  # Ambil kata-kata kunci untuk setiap topik
  terms = tfidf_vectorizer.get_feature_names_out()
  top_keywords = 1  # Jumlah kata kunci untuk setiap topik (HANYA BISA DIAMBIL 1)

  kata_kunci = []

  for i, topic in enumerate(lsa_model.components_):
      top_keywords_idx = topic.argsort()[-top_keywords:][::-1]
      keywords = [terms[key_idx] for key_idx in top_keywords_idx]
      kata_kunci.append(keywords[0])

  topik.append(kata_kunci)

df_temu['topik'] = topik

ValueError: n_components(5) must be <= n_features(4).

In [45]:
df_temu[['title', 'similaritas','abstract','keywords','topik']].sort_values(by=['similaritas'], ascending=False)

,title,similaritas,abstract,keywords,topik
1,Media Pembelajaran Digital sebagai Sumber Bela...,[[0.5970104031840774]],Kegiatan pembelajaran siswa SMA dan SMK mutlak...,NaN,"[pembelajaran, sejarah, siswa]"
6387,KEBUTUHAN MEDIA PEMBELAJARAN MAHASISWA: ANALIS...,[[0.5539309575943934]],ABSTRAK\n\nMahasiswa saat ini yang biasa diseb...,"Mahasiswa, Media Pembelajaran, Media Pembelaja...","[pembelajaran, media, mahasiswa]"
2489,COACHING CLINIC PEMBUATAN KONTEN DIGITAL MATA ...,[[0.5259985263064901]],Kegiatan pelatihan pembuatan konten digital ma...,"konten digital, Gather.town, media digital, ko...","[digital, bahasa, inggris]"
7495,Media Pembelajaran Sejarah,[[0.5031026124151314]],NaN,NaN,"[pembelajaran, media, sejarah]"
2490,COACHING CLINIC PEMBUATAN KONTEN DIGITAL MATA ...,[[0.48332430971587176]],Kegiatan pelatihan pembuatan konten digital ma...,NaN,"[bahasa, inggris, digital]"
...,...,...,...,...,...
5675,Deskripsi Self-Efficacy Berpikir Kritis Matema...,[[0.10039117569610857]],Penelitian kualitatif ini bertujuan untuk mend...,NaN,"[data, siswa, fficacy]"
14576,The Effectiveness of Guided Inquiry Learning M...,[[0.10025950960692029]],Penelitian ini bertujuan untuk mendeskripsikan...,"asam basa, kemampuan berpikir kritis, model pe...","[kelas, penelitian, kelas]"
366,Inovasi Program Penguatan Pendidikan Karakter ...,[[0.10019387053512613]],"Perkembangan zaman yang semakin modern, membaw...",NaN,"[data, media, penelitian]"
2194,Analisis Kesalahan Penggunaan Pronoms Relatifs...,[[0.10014038238353476]],Abstrak: Dewasa ini masih banyak siswa yang be...,"Pronoms Relatifs Simples, Test Essai, Semester...","[simples, relatifs, pronoms]"


# Test SCRIPT STKI

In [22]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import RegexpTokenizer
from tqdm import tqdm

def search(teks):
  df = pd.read_excel(r'D:\TEST Codingan\Sistem-Temu-Kembali-Informasi\Riset Jupiter Notebook\Data\df_final.xlsx')
  df = df.drop(columns='Unnamed: 0')

  teks = stem(teks)

  df_temu = []
  similaritas = []

  # Inisialisasi TF-IDF Vectorizer
  tfidf_vectorizer = TfidfVectorizer()

  for i in tqdm(range(len(df['text_stemmed']))):
    temp = df['text_stemmed'].iloc[i]

    # Transformasi teks menjadi vektor TF-IDF
    tfidf_matrix = tfidf_vectorizer.fit_transform([teks, temp])

    # Hitung similaritas kosinus antara kedua teks
    cosine_similarities = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])

    if cosine_similarities[0][0] > 0.1:
      df_temu.append(df.iloc[i])
      similaritas.append(float(cosine_similarities))

  df_temu = pd.DataFrame(df_temu)
  df_temu['similaritas'] = similaritas
  return tentukan_topik(df_temu.sort_values(by=['similaritas'], ascending=False))

def remove_stopwords(text):
  factory = StopWordRemoverFactory()
  stopwords = factory.get_stop_words()

  result = []
  for x in  text.split():
      if x not in stopwords:
          result.append(x)

  hasil_stopword = " ".join(result)

  return hasil_stopword

def stem(text):
  factory2 = StemmerFactory()
  stemmer = factory2.create_stemmer()
  return stemmer.stem(text)

def hapus_nan(dataset):
  dataset.fillna("", inplace=True) 
  return dataset

def tentukan_topik(dataset):
  tokenizer = RegexpTokenizer(r'\w+')
  topik = []

  for index in range(len(dataset)):
    # Ambil dataset
    teks = dataset['fulltext_stopword'].iloc[index]
    data = tokenizer.tokenize(teks)

    # Proses vektorisasi teks dengan TF-IDF
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(data)
    jumlah_feature = tfidf_matrix.shape

    # Terapkan LSA dengan TruncatedSVD
    if jumlah_feature[1] < 5:
      num_topics = jumlah_feature[1] # Jumlah topik yang ingin diidentifikasi
    else:
      num_topics = 5 # Jumlah topik yang ingin diidentifikasi
    lsa_model = TruncatedSVD(n_components=num_topics, random_state=42)
    lsa_model.fit_transform(tfidf_matrix)

    # Ambil kata-kata kunci untuk setiap topik
    terms = tfidf_vectorizer.get_feature_names_out()
    top_keywords = 1  # Jumlah kata kunci untuk setiap topik (HANYA BISA DIAMBIL 1)

    kata_kunci = []

    for i, topic in enumerate(lsa_model.components_):
        top_keywords_idx = topic.argsort()[-top_keywords:][::-1]
        keywords = [terms[key_idx] for key_idx in top_keywords_idx]
        kata_kunci.append(keywords[0])

    topik.append(kata_kunci)
  dataset['topik'] = topik
  dataset = dataset.drop(columns=[
     'text_stemmed',
     'fulltext_stopword', 
     'fulltext', 
     'language_accuracy', 
     'language', 
     'text_raw',
     'abstract', 
    #  'date',
    #  'keywords',
    #  'similaritas',
    #  'new_abstract',
    #  'publisher',
    #  'publication',
    #  'divisions'
    ])
  
  dataset = hapus_nan(dataset) #hapus nilai nan
  return convert_dataframe_to_list(dataset)
  # return convert_dataframe_to_dict(dataset)

def convert_dataframe_to_list(dataset):
  dataset = dataset.T
  hasil_pencarian = []
  for i in dataset.columns.values.tolist():
    hasil_pencarian.append(dataset[i].to_list())
  return hasil_pencarian

def convert_dataframe_to_dict(dataset):
  dataset = dataset.T
  hasil_pencarian = dataset.to_dict()
  return hasil_pencarian

In [23]:
df = search('hutan kawasan lindung')
df

100%|██████████| 15867/15867 [01:04<00:00, 245.03it/s]


[["['FP5']",
  'KEANEKARAGAMAN JENIS POHON SEBAGAI SALAH SATU INDIKATOR KESEHATAN HUTAN LINDUNG (Studi Kasus di Kawasan Hutan Lindung yang Dikelola oleh HKm Beringin Jaya)',
  "['SD']",
  'belantara',
  'Program Studi Kehutanan Fakultas Pertanian Universitas Mataram',
  '[{\'name\': {\'family\': "Safe\'i", \'given\': \'Rahmat\', \'lineage\': None, \'honourific\': None}, \'id\': \'rahmat.safei@fp.unila.ac.id\'}, {\'id\': \'hari.kaskoyo@fp.unila.ac.id\', \'name\': {\'honourific\': None, \'lineage\': None, \'given\': \'Hari\', \'family\': \'Kaskoyo\'}}, {\'name\': {\'honourific\': None, \'given\': \'Arief\', \'lineage\': None, \'family\': \'Darmawan\'}, \'id\': \'arief.darmawan@fp.unila.ac.id\'}, {\'id\': None, \'name\': {\'lineage\': None, \'given\': \'Fansuri Fikri\', \'family\': \'Haikal\', \'honourific\': None}}]',
  datetime.datetime(2021, 3, 1, 0, 0),
  '',
  'http://repository.lppm.unila.ac.id/id/eprint/29829',
  'Hutan lindung menjadi suatu kawasan hutan dengan fungsi pokok sebaga